In [2]:
from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

load_dotenv(find_dotenv("../config/.env"))
#openai.api_key = os.getenv("OPENAI_API_KEY")

True

In [3]:
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

In [4]:
template = """ 
You are AN-FactChecker, a bot which acts as a 
You will be provided pieces of context, of which are absolutely correct.
In addition to that, you will also be provided with a claim, that might or might not be factually correct (in relation to the context that is provided).
Your job will be to determine whether the claim is true or not. 

If the claim is true, you shall reply with 
"""

In [5]:
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(llm,
                                chain_type='stuff',
                                retriever=vectorstore.as_retriever(),
                                chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})